## Some examples on AD

Lets consider the Speelpenning problem $f(x)=\prod_{i=1}^{n} x_i$
We can code this in Julia in a variety of ways:

In [ ]:
f(x) = prod(x)
g(x) = reduce(*,x)

Julia Doc says prod should preferred over reduce(*).

Lets give it a try:

In [ ]:
x = collect(range(1.0,5.0)) #numbers of 1 to 5 (float)
f(x)

Now lets try to calculate the Gradient.
Choose one of many AD solutions already in the Julia ecosystem.

Lets start with forward (Tangent) mode:

In [ ]:
import ForwardDiff
ForwardDiff.gradient(f,x)

What about reverse mode?

In [ ]:
import Zygote
Zygote.gradient(f,x)

Maybe also FiniteDifferences just to be sure?

In [2]:
import FiniteDifferences
FiniteDifferences.grad(FiniteDifferences.central_fdm(2, 1), f, x) # 2nd order accurate 1st derivative approx

LoadError: UndefVarError: f not defined

Interjection: FiniteDifferences can do Richardson extrapolation out of the box:

In [9]:
FiniteDifferences.central_fdm(2,1)(x->sin(x),1.0) - cos(1.0) #regular central FD

-2.8588625911041277e-9

In [10]:
FiniteDifferences.extrapolate_fdm(FiniteDifferences.central_fdm(2,1),x->sin(x),1.0)[1] - cos(1.0)

1.6653345369377348e-14

For scalar functions Zygote implements the adjoint operator '

In [ ]:
import Zygote
using Plots
plot(x->sin(x))
plot!(x->sin'(x))

Note that the ' operation actually immediately creates machine code for the derivative

In [ ]:
s(x)=x^2
@code_llvm debuginfo=:none s'(3.0)

## Limitations of Adjoint AD in Zygote

No mutation within Arrays allowed!
Consider the following contrived example of constructing a matrix.

In [ ]:
function mat_sum(p)
    A=zeros(3,3)
    for i in 1:3
        for j in 1:3
            A[i,j] = j*p
        end
    end
    sum(A)
end

Calculating the derivative with Zygote will not work!

In [ ]:
mat_sum'(1.0)

If code is written with list comprehensions or other "pythonic" looking things it will usually work with Zygote well. However this prevents some vectorization techniques of Julia (e.g. element wise assignment .=) is also not supported.
If one is not very used to programming this way this becomes unmaintainable really fast.

In [ ]:
function mat_sum_no_mutation(p)
    A=mapreduce(i->ones(3)*i*p, hcat, 1:3) # iterate from 1 to 3, build columns of matrix and concats horizontally
    sum(A)
end
mat_sum_no_mutation'(1.0)

## Specifying custom adjoints
If altering the primal is not feasible or if structure can be exploited we can specify custom rules for the reversal.
Lets consider the common problem of the solution of a linear equation system embedded in some other form of computation:

In [ ]:
import Random
import LinearAlgebra as LA
import Zygote
import ForwardDiff
import ChainRulesCore

function solve(A,b)
    C = LA.factorize(A)
    x = C \ b
end

Factorize will decide which Factorization to use based of the Type of Matrix A provided (LU for general Matrix, Cholesky for SPD etc).

Lets add a custom rrule for Symmetric or SPD Matrices (we can reuse the existing Factorizations)
Note that the ∂ symbol is only convention here. It could be called anything. Order of incoming and outgoint adjoints is determined by the argument position in the primal routines.

In [ ]:
function ChainRulesCore.rrule(::typeof(solve), A::T, b::AbstractVector) where T <: Union{LA.Hermitian,LA.Symmetric}
    @info "Chainrule for symmetric Matrix solve triggered."
    C = LA.factorize(A) # duplication of primal so we can use C later
    x = C \ b
    function solve_pullback(∂x) # incoming adjoints
        ∂b = C \ ∂x    # reuse existing factorization to solve C*∂b=∂x
        ∂A = -x * ∂b'  # calculate outer product
        return ChainRulesCore.NoTangent(), ∂A, ∂b # return adjoints for ∂A, ∂b
    end
    return x, solve_pullback # return primal result x, pullback will be called during reversal
end

And another rule for general matrices (we discard the factorization and build a new one for A^T):

In [ ]:
function ChainRulesCore.rrule(::typeof(solve), A::LA.Matrix, b::AbstractVector)
    @info "Chainrule for general matrix solve triggered."
    x = solve(A,b) # reuse primal implementation
    function solve_pullback(∂x)
        ∂b = solve(A',∂x) # reuse primal implementation
        ∂A = -x * ∂b'
        return ChainRulesCore.NoTangent(), ∂A, ∂b
    end
    return x, solve_pullback
end

Construct a function $\mathbb{R}^n \rightarrow \mathbb{R}$ which uses solve()
(we fix the random seed such that gradient is reproducible)

In [ ]:
ChainRulesCore.@non_differentiable Random.seed!(::Any...)
function foo(p)
    Random.seed!(0)
    n = length(p)
    A = randn(n,n)
    A = LA.Hermitian(A + A' + LA.Diagonal(p))
    b = randn(n)
    x = solve(A,b)
    #println(x)
    sum(x)
end
p = randn(10)

In [ ]:
Zygote.gradient(foo,p)


In [ ]:
ForwardDiff.gradient(foo,p)